In [ ]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
import pandas as pd
import numpy as np
import os
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import re
import glob
from pathlib import Path
from helpers_frame_extraction import *

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
csv_path = Path('./csv_for_frame_extraction/')

In [ ]:
csv_paths = list(csv_path.glob('*')); csv_paths

In [ ]:
events_csv = pd.read_csv(str(csv_paths[0]))

# Take Folders one by one so that no frames from different folders are mixed

In [ ]:
folders = events_csv.folder.unique(); folders

In [ ]:
df_vid1 = events_csv[events_csv['folder'] == 'DATA_20120924003102032']

# Choosing columns of interest

In [ ]:
df_vid1 = columns_of_interest(df_vid1)

In [ ]:
df_vid1.head()

# Extract for all 3 cameras

In [ ]:
folder_path = Path('/media/data/astamoulakatos/Survey-2-2012/Project 1/IC2/KP130.417-145.077_A')

In [ ]:
vid1_path = folder_path/folders[0]

In [ ]:
vid1_path

In [ ]:
videos = list(vid1_path.glob('*.mpg')); videos

In [ ]:
if 'Ch2' in videos[0].parts[-1]:
    print('yes')

# Fill in KP values and sort by KP

In [ ]:
df_vid1 = fill_in_KP(df_vid1)

# Add event for start of video! (offset and observation code)

In [ ]:
df_vid1 = add_event_for_start(df_vid1)

In [ ]:
df_vid1.head()

# Add event for end of video! (observation code, the frames of videos should be counted for offsets)

In [ ]:
df_vid1 = add_event_for_end(df_vid1, videos)

In [ ]:
df_vid1.tail()

# Testing frame Extraction